# C# Lowering and State machines

## Lowering

To understand how C# manages Tasks to be non-blocking, first you need to understand `lowering`.  
I couldn't find any docs on this. Lowering basically is the compiler changing complex C# code into simpler instructions.  

The C# compiler will transform you code into `IL Code` or intermediate language code.  
This is a language your JIT compiler understand and is something close to machine code but also not really.  
It's closer to Java's JAR file.

Because you don't want to have to update the runtime every time you add new syntax to C# the compiler lowers your code into something more lower level.  
For instance, consider a foreach loop:

In [5]:
using System;
using System.Collections.Generic;

var exampleList = new List<int>{ 1, 2, 3 };

foreach (var item in exampleList)
{
    Console.Write(item.ToString() + " ");
}

1 2 3 

The compiler does not know what a foreach is, so then how does this work.  
We can rewrite this as follows:

In [6]:
using System;
using System.Collections.Generic;

var exampleList = new List<int>{ 1, 2, 3 };

var exampleEnumerator = exampleList.GetEnumerator();
while (exampleEnumerator.MoveNext())
{
    var item = exampleEnumerator.Current;
    Console.Write(item.ToString() + " ");
}

1 2 3 

In fact this is what the compiler lowers a foreach loop into.  
See [this example](https://sharplab.io/#gist:c45657e27dc3496d18f29e1f1efa66e7) on [sharplabs.io] to see what it actually does.

# State machines

To understand how async works and give some more insights on how these enumerables work in DotNet consider the following code:

In [2]:
using System;
using System.Collections.Generic;

var exampleList = new List<int>{ 1, 2, 3 };

foreach (var item in exampleList)
{
    Console.Write(item.ToString() + " ");
}

static IEnumerable<int> ExampleList() {
    yield return 1;        
    yield return 2;        
    yield return 3;
}

1 2 3 

In this example the `yield return` statement doesn't do much. Normally this may contain some logic in between it to determine when and what to yield etc.  
When you inspect [this example](https://sharplab.io/#gist:93218f4fbb0d520690bcc629f8a284fe) in sharplabs you can see what kind of code is generated.  
This is how C# orchestrates yielding internally.

So why is this important?  
This is the simples example of a state machine to be found within compiler generated code.  
If you inspect the `MoveNext()` you can get an idea of how this works:

In [6]:
private int __state;
private int __current;

private bool MoveNext()
{
    switch (__state)
    {
        default:
            return false;
        case 0:
            __state = -1;
            __current = 1;
            __state = 1;
            return true;
        case 1:
            __state = -1;
            __current = 2;
            __state = 2;
            return true;
        case 2:
            __state = -1;
            __current = 3;
            __state = 3;
            return true;
        case 3:
            __state = -1;
            return false;
    }
}

There's a log going on around this so let's zoom in on the `MoveNext()`.  
We gave it 3 values, and the compiler generates a switch statement with 4 cases,   
1 for each int, and a last one with a state to indicate we're at the end of the collection.